In [1]:
import numpy as np
import pandas as pd
# from calibration.Calibration import Calibration 


In [2]:
# load rankings
reco_matrix = np.load("model_reco/vaecf/reco_matrix_vaecf_ml100k_100.npy")
print(reco_matrix.shape)

(943, 100)


In [3]:
import pickle

with open("model_reco/vaecf/score_dicts_vaecf_ml100k.pkl", "rb") as file:
    scores = pickle.load(file)
scores.__len__()


943

In [4]:
movies = pd.read_csv(
    "./data/ml-100K/i_id_mapping_genre.csv",
    sep="\t",
    names=[
        "item_id",
        "Name",
        "genres",
        "itemID"
    ],
    header=0,
    # encoding="latin-1",
)
movies = movies.drop(columns=["item_id"])
movies = movies.sort_values(by="itemID")
unique_genres = [
    "Action",
    "Thriller",
    "Romance",
    "Western",
    "Children's",
    "Mystery",
    "Fantasy",
    "Film-Noir",
    "Documentary",
    "Comedy",
    "Adventure",
    "Sci-Fi",
    "Horror",
    "Crime",
    "Musical",
    "War",
    "Animation",
    "Drama",
]
for genre in unique_genres:
    movies[genre] = 0
    
for index, row in movies.iterrows():
    genres = row["genres"].split("|")
    for genre in genres:
        movies.at[index, genre] = 1


movies

,Name,genres,itemID,Action,Thriller,Romance,Western,Children's,Mystery,Fantasy,...,Documentary,Comedy,Adventure,Sci-Fi,Horror,Crime,Musical,War,Animation,Drama
0,Kolya (1996),Comedy,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,L.A. Confidential (1997),Thriller|Mystery|Film-Noir|Crime,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,Heavyweights (1994),Children's|Comedy,2,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Legends of the Fall (1994),Romance|Western|War|Drama,3,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,Jackie Brown (1997),Crime|Drama,4,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343,American Buffalo (1996),Drama,1343,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1344,A Chef in Love (1996),Comedy,1344,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1345,Nénette et Boni (1996),Drama,1345,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1346,Hollow Reed (1996),Drama,1346,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
users = pd.read_csv("./data/ml-100k/u_id_mapping_demographic_.csv", sep="\t")
users = users.sort_values(by="userID")
gender_map = {"M": 0, "F": 1}
users["Gender"] = users["Gender"].map(gender_map)
user_features_numpy = users.to_numpy()
users


,user_id,Age,Gender,Occupation,ZipCode,userID,Occupation_Code
195,196,49,0,writer,55105,0,20
185,186,39,1,executive,00000,1,6
21,22,25,0,writer,40206,2,20
243,244,28,0,technician,80525,3,19
165,166,47,0,educator,55113,4,3
...,...,...,...,...,...,...,...
938,939,26,1,student,33319,938,18
935,936,24,0,other,32789,939,13
929,930,28,1,scientist,07310,940,17
919,920,30,1,artist,90008,941,1


In [6]:
item_ids = movies["itemID"].to_numpy()
item_ids

array([   0,    1,    2, ..., 1345, 1346, 1347])

In [18]:
# tahsin kheya
# last modified 07/01/2025
import pandas as pd
import os
import numpy as np

# import torch
# import time
from logging import getLogger
import random
import numpy as np
from scipy.stats import entropy
# from mpi4py import MPI

# comm = MPI.COMM_WORLD
rank = 0


class GreedyCalibration(object):
    def __init__(self, config, movies, top_k, unique_genres, users, sensitive_attr, beta):
        self.top_k = top_k
        self.users_df = users
        self.beta = beta
        self.actual_genre_dist = pd.read_csv(
            os.path.join(config["user_genre_dist_file"]),
            sep="\t",
        )
        self.actual_genre_dist = self.actual_genre_dist.sort_values(by="userID")
        self.unique_genres = unique_genres
        self.item_df = movies
        self.sensitive_attr = sensitive_attr
        self.sensitive_compare_dist = []
        self.actual_distribution_sensitive = []

    def get_recom_distribution(self, reco, uid, compare_dist, alpha):
        reco = np.array(reco)
        df_reco = pd.DataFrame(
            {
                "userID": np.repeat(uid, len(reco)),
                "itemID": reco.flatten(),
                "rank": np.tile(np.arange(1, len(reco) + 1), 1),
            }
        )
        df_reco["weight_factor"] = 1 / (df_reco["rank"]) ** 0.1
        merged_df = pd.merge(df_reco, self.item_df, on="itemID", how="inner")

        merged_df[self.unique_genres] = merged_df[self.unique_genres].div(
            merged_df[self.unique_genres].sum(axis=1), axis=0
        )
        for i, genre in enumerate(self.unique_genres):
            merged_df[genre] = (1 - alpha) * merged_df[genre] + alpha * compare_dist[i]
       
        merged_df[self.unique_genres] = (
            merged_df["weight_factor"].values[:, None] * merged_df[self.unique_genres]
        )
        # for i, genre in enumerate(self.unique_genres):
        #     merged_df[genre] = (1 - alpha) * merged_df[genre] + alpha * compare_dist[i]

        summed_genre = (
            merged_df.groupby("userID")[self.unique_genres].sum().reset_index()
        )

        return summed_genre[self.unique_genres].to_numpy()

    def compute_diversity_score(self, reco_items, uid, scores, b):
        alpha = 0.01
        sum_score = 0
        current_user_sensitive_attr = self.users_df.loc[
            self.users_df["userID"] == uid, self.sensitive_attr
        ].item()
        compare_dist = self.sensitive_compare_dist[current_user_sensitive_attr]
        reco_dist = self.get_recom_distribution(reco_items, uid, compare_dist, alpha)[
            0
        ]  # sum wr(i)q˜(д|i),
        
        reco_dist = np.log(reco_dist)  # log sum wr(i)q˜(д|i),
        faireness_term = np.sum(compare_dist * reco_dist)

        for r in range(len(reco_items)):
            sum_score += scores[reco_items[r]]

        return (sum_score, faireness_term)

    def get_improved_reco(self, top_items, items, scores):
        self.get_sensitive_genre_dist()

        return self.get_new_recommendations(
            reco=top_items, scores=scores, all_items=items
        )

    def get_sensitive_genre_dist(self):
        actual_dist_sensitive = pd.merge(
            self.actual_genre_dist, self.users_df, on="userID"
        )
        sensitive_genre_weights_a = actual_dist_sensitive.groupby(self.sensitive_attr)[
            self.unique_genres
        ].mean()

        self.actual_distribution_sensitive = sensitive_genre_weights_a.sort_index()

        sensitive_compare_dist = self.actual_distribution_sensitive.sum()

        self.sensitive_compare_dist = self.actual_distribution_sensitive.apply(
            lambda r: sensitive_compare_dist - r, axis=1
        )
        self.sensitive_compare_dist = (
            self.sensitive_compare_dist.sort_index().to_numpy()
        )

    def normalize_scores(self, diversity_scores, b):
        # print(f"beta - {b}")
        
        filtered_scores = [x for x in diversity_scores if isinstance(x, tuple) and len(x) == 2]
        
        min_score = min(filtered_scores, key=lambda x: x[0])[0]
        max_score = max(filtered_scores, key=lambda x: x[0])[0]

        min_fairness = min(filtered_scores, key=lambda x: x[1])[1]
        max_fairness = max(filtered_scores, key=lambda x: x[1])[1]

        all_scores = []
        for i in range(diversity_scores.__len__()):
            if diversity_scores[i] != 0:
                score = (diversity_scores[i][0]- min_score) /(max_score-min_score)
                fairness_term = (diversity_scores[i][1]-min_fairness)/(max_fairness-min_fairness)
                # print(
                #     f"score {score} ft {fairness_term} total {(1 - b) * score + b * fairness_term}"
                # )
                all_scores.append((1 - b) * score + b * fairness_term)
            else:
                # print(f"index {i}")
                all_scores.append(-9999)

        return all_scores

    def get_new_recommendations(self, reco, scores, all_items):
        """reco is 6040x50 and scores is 6040x3416"""
        b = self.beta  # beta for the fairness term
        # print(f"beta {b}")
        all_users = []
        top_k = self.top_k
        num_users = len(scores)
        # for u in range(rank*2,rank*2+2):
        upper_bound = min(num_users, rank * 25 + 25)
        
        for u in range(0, num_users):
            # remaining_items = list(range(20))
            remaining_items = reco[u]
            # print(remaining_items)
            u_calibrated = []
            for k in range(top_k):
                diversity_scores = [
                    self.compute_diversity_score(u_calibrated + [i], u, scores[u], b)
                    for i in remaining_items
                ]
                # print(f"before {remaining_items} len {remaining_items.__len__()}")
                
                norm_diversity_scores = self.normalize_scores(diversity_scores, b)
                max_index = np.argmax(norm_diversity_scores)
                best_item = remaining_items[max_index]
                # print(f"best item {best_item}")
                # popped_element = arr[index_to_pop     ]
                # new_arr = np.delete(arr, index_to_pop)
                
                u_calibrated.append(best_item)
                remaining_items=np.delete(remaining_items, max_index)
                # print(f"after {remaining_items} len {remaining_items.__len__()}")
                # print(u_calibrated)


                
                # remaining_items.pop(max_index)
                # print(u_calibrated)
            print(f"user {u} u_calibrate {u_calibrated}")

            all_users.append(u_calibrated)

        return np.array(all_users)


In [ ]:
config = {"user_genre_dist_file": "./data/ml-100k/pgui.csv"}
sensitive_attr = "Gender"
# movies = movie df
calibration = GreedyCalibration(config, movies, 20, unique_genres, users, sensitive_attr, 0.5)
retVal = calibration.get_improved_reco(reco_matrix, list(item_ids), scores)

user 0 u_calibrate [289, 49, 357, 489, 189, 60, 24, 157, 1, 89, 360, 645, 95, 52, 133, 148, 31, 139, 101, 200]
user 1 u_calibrate [364, 52, 95, 136, 49, 24, 60, 549, 402, 77, 140, 157, 86, 23, 25, 641, 340, 209, 196, 278]
user 2 u_calibrate [82, 114, 748, 161, 26, 142, 526, 136, 201, 216, 247, 454, 407, 31, 179, 57, 368, 208, 231, 52]
user 3 u_calibrate [161, 24, 49, 52, 31, 136, 179, 256, 68, 307, 77, 367, 357, 216, 364, 402, 57, 86, 140, 209]
user 4 u_calibrate [710, 95, 645, 157, 1, 289, 60, 40, 360, 489, 98, 742, 139, 403, 117, 175, 104, 72, 280, 130]
user 5 u_calibrate [357, 49, 24, 31, 321, 254, 161, 216, 256, 367, 100, 166, 239, 189, 191, 240, 469, 52, 329, 217]
user 6 u_calibrate [49, 357, 24, 189, 77, 52, 31, 95, 136, 209, 60, 500, 402, 216, 12, 289, 103, 68, 101, 686]
user 7 u_calibrate [357, 49, 24, 31, 52, 254, 367, 256, 189, 161, 216, 101, 209, 321, 200, 307, 77, 156, 95, 239]
user 8 u_calibrate [289, 49, 357, 489, 189, 31, 273, 36, 100, 256, 240, 108, 24, 101, 89, 618, 23

In [21]:
np.save("reco_mat_test", retVal)

In [28]:
retVal.__len__()

943

In [40]:
p = np.array([0.1, 0.3, 0.6])
q = np.array([0.3, 0.4, 0.3])
# q˜(д|u) = (1 − α) · q(д|u) + α · p(д|u)
q_t = (1 - 0.01) * q + 0.01 * p
print(q_t)

[0.298 0.399 0.303]


In [34]:
movies[793:794]

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,itemID,genres
265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,793,


In [35]:
movies[movies["genres"]==""]

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,itemID,genres
265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,793,


In [34]:
scores[0][1]

4.4314069747924805

In [ ]:
sum(
    [
        0.0,
        0.01923077,
        0.0,
        0.11965812,
        0.0,
        0.02136753,
        0.0,
        0.00641026,
        0.0,
        0.02564101,
        0.5769231,
        0.02777778,
        0.01282051,
        0.0,
        0.0,
        0.0,
        0.01495726,
        0.0,
        0.17521365,
    ]
)

0.99999999

In [ ]:
item_index = movies.index.get_loc(0)
item_index

24

In [ ]:
reco = [1, 23, 4]
reco_2 = np.array(reco)
reco_2.flatten()

array([ 1, 23,  4])

In [ ]:
nonzero_indices = np.where(p_dist != 0)

In [ ]:
nonzero_indices

(array([0, 2]),)

In [1]:
movies

NameError: name 'movies' is not defined

In [25]:
actual_genre_dist = pd.read_csv(
            os.path.join(config["user_genre_dist_file"]),
            sep="\t",
        )
actual_genre_dist = actual_genre_dist.sort_values(by="userID")
x = pd.merge(
            actual_genre_dist, users, on="userID"
        )
x = x.groupby(sensitive_attr)[
            unique_genres
        ].mean()
x

,Action,Thriller,Romance,Western,Children's,Mystery,Fantasy,Film-Noir,Documentary,Comedy,Adventure,Sci-Fi,Horror,Crime,Musical,War,Animation,Drama
Occupation_Code,,,,,,,,,,,,,,,,,,
0,0.095032,0.101878,0.087882,0.006552,0.028753,0.025823,0.002530,0.005189,0.004081,0.173096,0.044490,0.041599,0.027925,0.033555,0.018150,0.035021,0.011738,0.256631
1,0.073128,0.088305,0.094621,0.007067,0.019756,0.027514,0.002664,0.007762,0.008769,0.168835,0.036485,0.038294,0.021778,0.032194,0.017760,0.041123,0.009702,0.304208
2,0.086279,0.113704,0.074507,0.004721,0.020429,0.035599,0.002058,0.011593,0.014286,0.153455,0.038676,0.048490,0.035978,0.039742,0.018879,0.030544,0.013597,0.257239
3,0.091096,0.100403,0.093103,0.006897,0.020257,0.025494,0.003149,0.006884,0.004571,0.185989,0.041992,0.042147,0.026972,0.034402,0.016116,0.037194,0.007642,0.255691
6,0.090982,0.118551,0.091389,0.004879,0.022348,0.026883,0.003624,0.006494,0.009669,0.157420,0.042403,0.033956,0.017687,0.031893,0.016840,0.037910,0.009599,0.277473
7,0.093444,0.111974,0.078046,0.005952,0.024510,0.033220,0.003652,0.005788,0.003601,0.176895,0.043949,0.044911,0.025041,0.034629,0.014947,0.034374,0.010732,0.254336
9,0.122219,0.157505,0.063421,0.000000,0.029517,0.054983,0.003323,0.004210,0.006638,0.178245,0.040776,0.038298,0.016817,0.039565,0.012154,0.028524,0.005396,0.198409
10,0.108874,0.107036,0.083767,0.006704,0.027948,0.022366,0.003665,0.004912,0.004380,0.183248,0.050137,0.051949,0.035274,0.033230,0.016439,0.030071,0.013954,0.216026
11,0.084759,0.103690,0.099115,0.008908,0.023498,0.031865,0.001846,0.009614,0.007435,0.192014,0.032027,0.035946,0.017774,0.030509,0.020561,0.038333,0.008573,0.253533


In [46]:
total_sum = x.sum()
total_sum

Action         1.414103
Thriller       1.629980
Romance        1.250090
Western        0.103136
Children's     0.350728
Mystery        0.459053
Fantasy        0.044175
Film-Noir      0.111304
Documentary    0.100646
Comedy         2.568908
Adventure      0.630022
Sci-Fi         0.661128
Horror         0.382142
Crime          0.530740
Musical        0.252659
War            0.550151
Animation      0.150592
Drama          3.809281
dtype: float64

In [27]:

sum_except_self = x.apply(lambda row: total_sum - row, axis=1)
sum_except_self

,Action,Thriller,Romance,Western,Children's,Mystery,Fantasy,Film-Noir,Documentary,Comedy,Adventure,Sci-Fi,Horror,Crime,Musical,War,Animation,Drama
Occupation_Code,,,,,,,,,,,,,,,,,,
0,1.319071,1.528102,1.162207,0.096584,0.321974,0.433231,0.041644,0.106116,0.096565,2.395812,0.585533,0.619529,0.354217,0.497184,0.234509,0.515130,0.138853,3.552651
1,1.340975,1.541675,1.155468,0.096069,0.330972,0.431539,0.041511,0.103543,0.091878,2.400073,0.593538,0.622834,0.360364,0.498545,0.234899,0.509028,0.140890,3.505073
2,1.327824,1.516276,1.175583,0.098415,0.330298,0.423454,0.042117,0.099711,0.086361,2.415452,0.591346,0.612638,0.346164,0.490998,0.233779,0.519607,0.136995,3.552043
3,1.323007,1.529577,1.156987,0.096239,0.330470,0.433560,0.041026,0.104421,0.096076,2.382919,0.588030,0.618981,0.355170,0.496337,0.236543,0.512957,0.142949,3.553590
6,1.323121,1.511429,1.158701,0.098257,0.328380,0.432171,0.040551,0.104811,0.090978,2.411487,0.587620,0.627172,0.364455,0.498846,0.235818,0.512241,0.140992,3.531808
7,1.320659,1.518005,1.172044,0.097185,0.326217,0.425833,0.040523,0.105517,0.097045,2.392013,0.586073,0.616217,0.357101,0.496111,0.237712,0.515778,0.139860,3.554945
9,1.291884,1.472475,1.186669,0.103136,0.321210,0.404070,0.040851,0.107095,0.094009,2.390663,0.589247,0.622830,0.365325,0.491175,0.240504,0.521627,0.145195,3.610873
10,1.305229,1.522944,1.166323,0.096432,0.322780,0.436687,0.040510,0.106392,0.096267,2.385659,0.579885,0.609179,0.346868,0.497509,0.236220,0.520080,0.136638,3.593255
11,1.329344,1.526290,1.150974,0.094228,0.327229,0.427188,0.042329,0.101691,0.093211,2.376894,0.597995,0.625182,0.364368,0.500231,0.232098,0.511818,0.142019,3.555748


In [32]:

sum_except_self.sort_index(ascending=True).to_numpy()[0]

array([1.31907121, 1.52810153, 1.16220731, 0.09658408, 0.32197414,
       0.43323066, 0.0416444 , 0.10611577, 0.09656508, 2.39581206,
       0.58553256, 0.61952947, 0.35421677, 0.49718446, 0.23450911,
       0.5151305 , 0.13885339, 3.55265061])

In [37]:
print(int(users[users["userID"]==1]["Occupation_Code"]))

7


/var/folders/x3/dln8h72x52l1m7szndwhlw6r0000gn/T/ipykernel_13180/2629660332.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  print(int(users[users["userID"]==1]["Occupation_Code"]))


In [39]:
occupation_code = users.loc[users["userID"] == 1, "Occupation_Code"].item()
occupation_code


7

In [83]:
users[users["Occupation_Code"]==4]

,user_id,Age,Gender,Occupation,ZipCode,userID,Occupation_Code


In [18]:
users

,user_id,Age,Gender,Occupation,ZipCode,userID,Occupation_Code
195,196,49,0,writer,55105,0,20
185,186,39,1,executive,00000,1,6
21,22,25,0,writer,40206,2,20
243,244,28,0,technician,80525,3,19
165,166,47,0,educator,55113,4,3
...,...,...,...,...,...,...,...
938,939,26,1,student,33319,938,18
935,936,24,0,other,32789,939,13
929,930,28,1,scientist,07310,940,17
919,920,30,1,artist,90008,941,1


In [17]:
0.5*0.99+0.01*0.26886745

0.4976886745

In [13]:
for i, genre in enumerate(unique_genres):
    print(i)
    print(genre)

0
Action
1
Thriller
2
Romance
3
Western
4
Children's
5
Mystery
6
Fantasy
7
Film-Noir
8
Documentary
9
Comedy
10
Adventure
11
Sci-Fi
12
Horror
13
Crime
14
Musical
15
War
16
Animation
17
Drama


In [14]:
2*(0.2*0.01+0.99*0.02)

0.0436

In [15]:
(0.2*0.01+0.99*0.04)

0.041600000000000005

In [17]:
0.895958*0.000353

0.000316273174